In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/train.csv")

In [4]:
interactions = (
    train.groupby(["msno", "song_id"])
         .size()
         .reset_index(name="play_count")
)

In [5]:
user_codes = interactions["msno"].astype("category").cat.codes
item_codes = interactions["song_id"].astype("category").cat.codes

interactions["user_id"] = user_codes
interactions["item_id"] = item_codes


In [6]:
user_mapping = dict(enumerate(interactions["msno"].astype("category").cat.categories))
item_mapping = dict(enumerate(interactions["song_id"].astype("category").cat.categories))

In [7]:
from scipy.sparse import csr_matrix

In [8]:
user_item_matrix = csr_matrix(
    (
        interactions["play_count"],
        (interactions["user_id"], interactions["item_id"])
    )
)

In [9]:
user_item_matrix.shape

(30755, 359966)

In [10]:
from implicit.als import AlternatingLeastSquares

/Users/aditisonkar12/Desktop/Python Projects/Music-recommendation-system/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=20,
    random_state=42
)

In [12]:
als_model.fit(user_item_matrix.T)

/Users/aditisonkar12/Desktop/Python Projects/Music-recommendation-system/venv/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.04389619827270508 seconds
  warnings.warn(
100%|██████████| 20/20 [00:46<00:00,  2.30s/it]
